## Movie Recomendation System

Pada perkembangan zaman seperti sekarang ini banyak sekali platform penyedia layanan streaming film seperti netfix, disney+, dan lain sebagainya. hal ini menyebabkan permasalahan bagi pengguna untuk mencari film yang sesui dengan preferensi mereka.  Pengguna sering kali merasa kewalahan dan menghabiskan banyak waktu hanya untuk mencari film yang mereka sukai. Oleh karena itu, diperlukan sebuah sistem yang dapat membantu pengguna menemukan film yang tepat dengan lebih efisien dan akurat.

Sistem rekomendasi merupakan jawaban atas permasalahan diatas. Dengan memanfaatkan data seperti genre film yang telah ditonton pengguna system dapat memberikan rekomendasi film yang serupa dengan yang telah ditonton oleh pengguna. hal ini dapat mempersingkat waktu bagi pengguna untuk mencari film yang serupa dengan film yang telah di tonoton

In [127]:
import numpy as np
import pandas as pd

    Pada system ini menggunakan data dari netfix yang kita ambil dari website kaggle

In [128]:
df=pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


    Dalam pembuatan system ini hanya menggunakan kolom title yang berisi judul dan kolom listed_in yang berisi genre film

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


    Dalam kolom title dan listed_in tidak terdapat missing value

In [130]:
df[df.duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


    tidak terdapat kolom duplicate

In [131]:
df['listed_in'].head()

0                                        Documentaries
1      International TV Shows, TV Dramas, TV Mysteries
2    Crime TV Shows, International TV Shows, TV Act...
3                               Docuseries, Reality TV
4    International TV Shows, Romantic TV Shows, TV ...
Name: listed_in, dtype: object

### Data preprosesing

    Menghitung frequensi kata yang terdapat dalam data kemudian di lakukan fit terhadap kolom listed_in/genre

In [132]:
from sklearn.feature_extraction.text import CountVectorizer

tf =CountVectorizer()
tf_matrix = tf.fit_transform(df['listed_in'])
tf_matrix.shape

(8807, 45)

    Menghitung similarity dari kolom listed in yang telah di lakukan term frequensi dengan cosine similarity

In [133]:
from sklearn.metrics.pairwise import linear_kernel

similarity = linear_kernel(tf_matrix, tf_matrix)

    Membuat fungsi yang untuk merekomendasikan judul film yang memiliki genre yag sama dengan yang telah di tonton

In [134]:
def recommendations(title, similarity=similarity):
    # Get the index of the movie that matches the title
    try:
        idx = df[df['title'] == title].index[0]
        # mendapatkan similarity
        sim_score = list(enumerate(similarity[idx]))

        # mengurutkan film berdasarkan similarity
        sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)

        # mendapatkan 5 film yang memiliki similarty 
        sim_score = sim_score[1:6]

        # mendapatkan index dari movie
        index_movie = [i[0] for i in sim_score]

        # mengembalikan 5 film yang memiliki similarity 
        return df.iloc[index_movie][['title', 'type', 'country','listed_in']]
    
    except IndexError:
        print(f"Title '{title}' not found in the dataset.")
        return []
    


    Mendapatkan rekomendasi film 

In [142]:
recommendations('Dick Johnson Is Dead')

,title,type,country,listed_in
16,Europe's Most Dangerous Man: Otto Skorzeny in ...,Movie,NaN,"Documentaries, International Movies"
45,My Heroes Were Cowboys,Movie,NaN,Documentaries
68,Schumacher,Movie,NaN,"Documentaries, International Movies, Sports Mo..."
88,Blood Brothers: Malcolm X & Muhammad Ali,Movie,NaN,"Documentaries, Sports Movies"
91,The Women and the Murderer,Movie,France,"Documentaries, International Movies"
